In [1]:
import re
import codecs
import sys

In [2]:
from __future__ import print_function
def warning(*objs):
    print("WARNING: ", *objs, file=sys.stderr)

In [3]:
nomFichier="texte.txt"
try:
    fichier = codecs.open(nomFichier, "r", "utf-8")
except IOError:
    warning("could not open", nomFichier)
    sys.exit()
else:
    text = fichier.readlines()
    fichier.close()

In [4]:
ngram = 5
pc = [".", ",", ";", ":", "?", "!", "--", "..."]
normalText=[]
for line in text:
    for ponct in pc: 
        line=line.replace(ponct,"")
    line=line.replace("'"," ")
    line=line.replace("-"," ")
    line=re.sub('\s+',' ',line)
    normalText.append(line.lower())
#normalText

In [5]:
fich = normalText
dataVectors=[]
dataClasses=[]
for line in fich[:]:
    line = line.strip()
#    print (line)
    espace = line.count(" ")
    i = 0
    while i <= len(line) - ngram - espace:
        if line[i] != " ":
            lettres = 0
            lettreslocales = 0
            ngramLocal=[]
            j = 0
            type = ""
            while lettres < ngram:
                if line[i + j] != " ":
                    ngramLocal.append(line[i + j])
                    lettres += 1
                    lettreslocales += 1
                else:
                    type = type + str(lettreslocales) + "_"
                    lettreslocales = 0
                j += 1
            type = type + str(lettreslocales)
            dataVectors.append(ngramLocal)
            dataClasses.append(type)
        i = i + 1

In [6]:
testTexte=u"danscettehistoireilyaunloupunechêvreetunagneautoutsepassebientantqueleloupetlagneaunesontpasseuls"
ngramTest=[list(testTexte[i:i+5]) for i,x in enumerate(testTexte) if i<len(testTexte)-4]
ngramComplete=dataVectors+ngramTest
textComplete=["".join(x) for x in ngramComplete]
textTrain=["".join(x) for x in dataVectors]
textTest=["".join(x) for x in ngramTest]
print (textComplete[0],textTrain[0],textTest[0])

chapi chapi dansc


CountVectorizer permet de traiter le texte comme des données numériques  
- analyzer permet de définir le niveau de tokenization (word, char)
- ngram_range permet de définir les ngrams qui seront extraits (min, max)

La méthode fit_transform permet de déterminer les données numériques (features) qui seront utilisées pour représenter le texte.

Ensuite, on peut utiliser la méthode transform pour représenter un texte en utilisant ces features.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
transformVectors = CountVectorizer(analyzer=u'char',ngram_range=(3, 5))
skComplete = transformVectors.fit_transform(textComplete)

SVM est une des formes d'apprentissage
- gamma définit le nombre de features qui seront utilisées pour l'apprentissage
- C ???

In [8]:
from sklearn import svm
clf = svm.SVC(gamma=0.001, C=100.)

skTrain est la représentation numérique de textTrain

clf.fit permet de faire l'apprentissage du lien entre les données de skTrain et les étiquettes fournies dataClasses

In [9]:
%%time
skTrain=transformVectors.transform(textTrain)
clf.fit(skTrain, dataClasses)

CPU times: user 29min 41s, sys: 12.3 s, total: 29min 53s
Wall time: 30min 10s


In [178]:
#skTest2=transformVectors.transform(textTrain[:100])
#prediction2=clf.predict(skTest2)

skTest est la représentation numérique de textTest

clf.predict permet de faire la prédiction des étiquettes pour les données de skTest

In [10]:
skTest=transformVectors.transform(textTest)
prediction=clf.predict(skTest)

##interprétation des résultats

###Découpage des ngrams

découpage des ngrams de test en fonction des étiquettes => resultChunks

In [30]:
resultChunks=[]
for i,element in enumerate(prediction):
    chunks=[int(n) for n in element.split("_")]
#    print (chunks)
    debut=0
    elementChunks=[]
    for chunk in chunks:
        elementChunks.append(textTest[i][debut:debut+chunk])
#        print (textTest[i][debut:debut+chunk],end=" ")
        debut+=chunk
#    print ()
    resultChunks.append(elementChunks)
#print (resultChunks)

###Calcul des votes

calcul des votes pour les séparations entre mots

In [31]:
testSpaces={}
for i,chunks in enumerate(resultChunks):
    posChunk=0
    for chunk in chunks:
        if posChunk!=0: 
#            print (i+posChunk,chunk)
            if not i+posChunk in testSpaces:
                testSpaces[i+posChunk]=0
            testSpaces[i+posChunk]+=1
        posChunk+=len(chunk)

###Découpage de test

Séparation des mots

In [38]:
for i,lettre in enumerate(testTexte):
    if i in testSpaces:
        # choisir le seuil d'apparition des espaces
        # au dessus de deux types convergents : >2
        # on insère un espace
        if testSpaces[i]>2:
            print (" ",end="")
        # sinon, par exemple, on colle les lettres
        # on pourrait aussi mettre le nombre de votes convergents
        # pour voir ce que ça donne
        else:
            pass
#            print (testSpaces[i],end="")
    print (lettre,end="")

dans cette histoire il yaun loupune chêvre et unagne autout se passe bientant que le loupetlagne aune sont passeuls